# IsoNet
------
This will be used to create the model for predicting precipitable stable water isotopes. Only creates and exports the model and trains it. Is not used to predict.

Table of Contents:
1. [Importing Data](#importing-data)
2. [Data Preprocessing](#data-preprocessing)
3. [Model Creation](#model-creation)
4. [Model Training](#model-training)
5. [Model Export](#model-export)

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime
import matplotlib.pyplot as plt

# Tensorflow and keras libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

2024-03-22 23:59:28.524178: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 23:59:28.572595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 23:59:28.572637: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 23:59:28.573907: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-22 23:59:28.580873: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 23:59:28.581884: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Importing Data
Importing the data, and then splitting it into features and labels

In [2]:
# Load in the data, separate the features and labels
data = pd.read_csv(r'Isoscape_Data.csv')
data = data.drop(['H2avg', 'dex'], axis=1)

# Delete the rows with NaN values
data = data.dropna()

features = data.drop(['O18Avg', 'Station'], axis=1)
labels = data['O18Avg']

# Adjusting the date time to be int values where Janurary 1st, of year 1 is 1
features['Date'] = pd.to_datetime(features['Date'])
#features['Date'] = features['Date'].map(datetime.datetime.toordinal)

# Convert the date column into year and day of year 
features['Year'] = features['Date'].dt.year
features['Day'] = features['Date'].dt.dayofyear
features = features.drop(['Date'], axis=1)

numFeatures = features.shape[1]

features.columns

Index(['Lat', 'Long', 'Alt', 'Precipitation (kg/m^2/s)', 'Temperature (K)',
       'Year', 'Day'],
      dtype='object')

## Data Preprocessing
Preprocessing the data, including:
* Convert into numpy arrays
* Split into training and testing sets
* Convert into tensorflow Datasets
* Windowing the data (Skipping for now, will do later) 
* Standardizing the data

In [3]:
# Split the data into training and testing sets
splitIndex = int(0.8 * len(features))
xTrain = features[:splitIndex]
xTest = features[splitIndex:]
yTrain = labels[:splitIndex]
yTest = labels[splitIndex:]

# Scale the data
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

# Convert the data into numpy arrays
xTrain = np.array(xTrain)
yTrain = np.array(yTrain)
xVal = xTrain[:int(0.2 * len(xTrain))]
yVal = yTrain[:int(0.2 * len(yTrain))]
xTrain = xTrain[int(0.2 * len(xTrain)):]
yTrain = yTrain[int(0.2 * len(yTrain)):]

xTest = np.array(xTest)
yTest = np.array(yTest)


# Convert traindata into tensor dataset
trainData = tf.data.Dataset.from_tensor_slices((xTrain, yTrain))
valData = tf.data.Dataset.from_tensor_slices((xVal, yVal))

## Model Creation
Creating the model, including:
* Creating the model architecture
* Compiling the model

Model architecture:
* Input layer
* 1 LSTM layer
* 2 Dense layers
* Output layer

In [4]:
numNeuorns = 64
model = Sequential()
model.add(InputLayer(input_shape=(numFeatures, 1)))
model.add(LSTM(numNeuorns))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

## Model Training
Training and evaluating the model

In [5]:
es = EarlyStopping(monitor='val_loss', mode = 'min', patience=100, restore_best_weights=True)

batchSize = 32
model.fit(
    trainData.batch(batchSize),
    batch_size=batchSize,
    epochs=250,
    validation_data=valData.batch(batchSize),
    callbacks=[es]
)

Epoch 1/250
48/48 [==============================] - 4s 18ms/step - loss: 403.3085 - root_mean_squared_error: 20.0825 - mean_absolute_error: 17.5992 - val_loss: 172.9242 - val_root_mean_squared_error: 13.1501 - val_mean_absolute_error: 9.2365
Epoch 2/250
48/48 [==============================] - 0s 7ms/step - loss: 119.4431 - root_mean_squared_error: 10.9290 - mean_absolute_error: 8.5691 - val_loss: 65.2313 - val_root_mean_squared_error: 8.0766 - val_mean_absolute_error: 7.1121
Epoch 3/250
48/48 [==============================] - 0s 7ms/step - loss: 43.2764 - root_mean_squared_error: 6.5785 - mean_absolute_error: 5.3952 - val_loss: 49.7192 - val_root_mean_squared_error: 7.0512 - val_mean_absolute_error: 6.1533
Epoch 4/250
48/48 [==============================] - 0s 7ms/step - loss: 30.1315 - root_mean_squared_error: 5.4892 - mean_absolute_error: 4.4205 - val_loss: 46.1037 - val_root_mean_squared_error: 6.7900 - val_mean_absolute_error: 5.8666
Epoch 5/250
48/48 [=========================

## Model Export

In [6]:
# Printing how well the model did on the test data
print(model.evaluate(xTest, yTest))

15/15 [==============================] - 1s 5ms/step - loss: 14.1177 - root_mean_squared_error: 3.7573 - mean_absolute_error: 2.8663
[14.117661476135254, 3.757347583770752, 2.8662967681884766]


In [14]:
# Save the model
model.save('Models/UN-NAMED.keras')

In [7]:
# Create new pd DataFrame with the predictions and actual values including the features
predictions = model.predict(xTest)
df = features[splitIndex:]
df['Predictions'] = predictions
df['Actual'] = yTest

# Export the data to a csv file
df.to_csv('results_test.csv', index=False)

15/15 [==============================] - 0s 3ms/step


/tmp/ipykernel_1989/4244123162.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predictions'] = predictions
/tmp/ipykernel_1989/4244123162.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actual'] = yTest


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 25281 (98.75 KB)
Trainable params: 25281 (98.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
